In [3]:
import os
os.chdir('/Users/Joel/Desktop/ga/solar_energy_prediction')

In [4]:
import pandas as pd
import netCDF4 as nc
import numpy as np
train_data = pd.read_csv('solar/data/kaggle_solar/train.csv', index_col='Date', parse_dates=['Date'])
train_data.head(5).iloc[:,0:13]

,ACME,ADAX,ALTU,APAC,ARNE,BEAV,BESS,BIXB,BLAC,BOIS,BOWL,BREC,BRIS
Date,,,,,,,,,,,,,
1994-01-01,12384900,11930700,12116700,12301200,10706100,10116900,11487900,11182800,10848300,10225200,11374200,10335300,11119200
1994-01-02,11908500,9778500,10862700,11666400,8062500,9262800,9235200,3963300,3318300,11316600,8318700,4711500,5530500
1994-01-03,12470700,9771900,12627300,12782700,11618400,10789800,11895900,4512600,5266500,11916000,8594700,7239600,5596200
1994-01-04,12725400,6466800,13065300,12817500,12134400,11816700,12186600,3212700,8270100,11884200,5754900,8842500,4360500
1994-01-05,10894800,11545200,8060400,10379400,6918600,9936300,6411300,9566100,8009400,9288900,10971000,8810100,10572300


In [5]:
dswrf = list(nc.Dataset('solar/data/kaggle_solar/train/dswrf_sfc_latlon_subset_19940101_20071231.nc','r+').variables.values())[-1]
subset = dswrf[0:3,2:4,2:5,2:4,2:4]
nc_var = pd.DataFrame(np.reshape(subset,(np.shape(subset)[0], np.product(np.shape(subset)[1:]))))
nc_var

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,200.0,210.0,190.0,200.0,580.0,570.0,560.0,540.0,410.0,400.0,...,190.0,200.0,580.0,580.0,570.0,570.0,420.0,410.0,400.0,400.0
1,210.0,210.0,200.0,200.0,600.0,600.0,580.0,580.0,430.0,420.0,...,200.0,200.0,600.0,600.0,580.0,580.0,430.0,420.0,420.0,410.0
2,200.0,210.0,190.0,200.0,590.0,590.0,570.0,570.0,420.0,420.0,...,190.0,200.0,580.0,580.0,570.0,570.0,420.0,410.0,410.0,400.0


In [6]:
Xtrain_dir = 'solar/data/kaggle_solar/train/'
Xtest_dir = 'solar/data/kaggle_solar/test'
ytrain_file = 'solar/data/kaggle_solar/train.csv'
station_file = 'solar/data/kaggle_solar/station_info.csv'
import solar.wrangle.wrangle
import solar.wrangle.subset
import solar.wrangle.engineer
import solar.analyze.model
import solar.report.submission
import numpy as np
from solar.wrangle.wrangle import SolarData

station = ['ACME', 'BEAV']

train_dates = ['1994-01-01','1994-03-01']

test_dates = ['2008-01-01', '2008-01-05']


station_layout = True

var = ['dswrf_sfc', 'uswrf_sfc', 'tcolc_eatm', 'spfh_2m']

# Keep model 0 (the default model) as a column for each of the variables (aggregated over other dimensions)
model = [0, 1]

# Aggregate over all times
#times = [12, 15, 18, 21, 24]
times = [21, 24]

default_grid = {'type':'relative', 'axes':{'var':var, 'models':model, 'times':times,
                                          'station':station}}
just_grid = [default_grid]



%prun input_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, just_grid, 'local')

In [7]:
input_data[0].iloc[0:10, list(range(0,7)) + list(range(47, 51))]

values                                           \
models                      0                                            
var                 dswrf_sfc uswrf_sfc tcolc_eatm   spfh_2m dswrf_sfc   
lat_longs                1_SE      1_SE       1_SE      1_SE      2_SW   
times                      21        21         21        21        21   
train_dates station                                                      
1994-01-01  ACME        540.0     115.0     0.0014  0.003520     530.0   
            BEAV        520.0     122.0     0.0009  0.002320     510.0   
1994-01-02  ACME        530.0     112.0     0.0116  0.003760     500.0   
            BEAV        510.0     122.0     0.0037  0.002420     460.0   
1994-01-03  ACME        550.0     118.0     0.0002  0.002882     550.0   
            BEAV        520.0     124.0     0.0003  0.002292     520.0   
1994-01-04  ACME        560.0     120.0     0.0000  0.002690     550.0   
            BEAV        530.0     127.0     0.0021  0.001670     530.0   
1994-01-05  ACME        540.0     115.0     0.0327  0.003340     540.0   
            BEAV        430.0      94.0     0.0710  0.001710     460.0   

                                                                        \
models                                           1                       
var                 uswrf_sfc tcolc_eatm   spfh_2m dswrf_sfc uswrf_sfc   
lat_longs                2_SW       2_SW      4_NW      1_SE      1_SE   
times                      21         21        21        24        24   
train_dates station                                                      
1994-01-01  ACME        110.0     0.0000  0.003670     360.0      81.0   
            BEAV        118.0     0.0014  0.003070     350.0      84.0   
1994-01-02  ACME        100.0     0.0100  0.004430     350.0      78.0   
            BEAV        102.0     0.0013  0.002560     350.0      87.0   
1994-01-03  ACME        114.0     0.0030  0.002610     370.0      83.0   
            BEAV        119.0     0.0040  0.002520     350.0      87.0   
1994-01-04  ACME        114.0     0.0003  0.002668     370.0      85.0   
            BEAV        123.0     0.0043  0.002178     360.0      91.0   
1994-01-05  ACME        112.0     0.0094  0.003740     340.0      72.0   
            BEAV         99.0     0.1407  0.002670     300.0      69.0   

                                
models                          
var                 tcolc_eatm  
lat_longs                 1_SE  
times                       24  
train_dates station             
1994-01-01  ACME        0.0259  
            BEAV        0.0056  
1994-01-02  ACME        0.0006  
            BEAV        0.0015  
1994-01-03  ACME        0.0000  
            BEAV        0.0000  
1994-01-04  ACME        0.0001  
            BEAV        0.0022  
1994-01-05  ACME        0.0323  
            BEAV        0.0038

In [8]:
from solar.analyze.model import Model
from sklearn.linear_model import LinearRegression

error_formula = 'mean_absolute_error'
cv_splits = 10
njobs = 1
write = 'local'
%prun lin_model = Model.model(input_data, LinearRegression, {'fit_intercept': [True]}, cv_splits, \
                    error_formula, njobs, write)

                       values                                           \
models                      0                                            
var                 dswrf_sfc uswrf_sfc tcolc_eatm   spfh_2m dswrf_sfc   
lat_longs                1_SE      1_SE       1_SE      1_SE      2_SW   
times                      21        21         21        21        21   
train_dates station                                                      
1994-01-01  ACME        540.0     115.0     0.0014  0.003520     530.0   
            BEAV        520.0     122.0     0.0009  0.002320     510.0   
1994-01-02  ACME        530.0     112.0     0.0116  0.003760     500.0   
            BEAV        510.0     122.0     0.0037  0.002420     460.0   
1994-01-03  ACME        550.0     118.0     0.0002  0.002882     550.0   
            BEAV        520.0     124.0     0.0003  0.002292     520.0   
1994-01-04  ACME        560.0     120.0     0.0000  0.002690     550.0   
            BEAV        530.0     127.

In [19]:
lin_predicts = lin_model.predict(input_data[0].iloc[0:98,:])
yvals = input_data[1][0:98]
error_list = abs(yvals.values[:] - lin_predicts)

In [20]:
error_list[0:3,0:3]

array([[ 1080942.13057669,  1099780.02543831,   185962.60950302],
       [ 1187057.86942331,  1168219.97456169,  2082037.39049698],
       [  604542.13057669,   623380.02543831,   290437.39049698]])

In [22]:
yvals

total_solar
date       location             
1994-01-01 ACME         12384900
           BEAV         10116900
1994-01-02 ACME         11908500
           BEAV          9262800
1994-01-03 ACME         12470700
           BEAV         10789800
1994-01-04 ACME         12725400
           BEAV         11816700
1994-01-05 ACME         10894800
           BEAV          9936300
1994-01-06 ACME          6639000
           BEAV          2245200
1994-01-07 ACME         13244700
           BEAV          9877800
1994-01-08 ACME         12927900
           BEAV         11545200
1994-01-09 ACME         12600300
           BEAV         11295300
1994-01-10 ACME          6406500
           BEAV         10785000
1994-01-11 ACME         12743400
           BEAV         11424900
1994-01-12 ACME         10453500
           BEAV         11794200
1994-01-13 ACME         12985200
           BEAV         12149100
1994-01-14 ACME         13080000
           BEAV         12227700
1994-01-15 ACME         11826300
           BEAV         10290600
...                          ...
1994-02-04 ACME         10506600
           BEAV         14513400
1994-02-05 ACME         15325500
           BEAV         14849400
1994-02-06 ACME         15568500
           BEAV         14813100
1994-02-07 ACME          9486600
           BEAV          4878900
1994-02-08 ACME          3012000
           BEAV          6858900
1994-02-09 ACME          7137000
           BEAV         15905100
1994-02-10 ACME         15829800
           BEAV         15714300
1994-02-11 ACME         16185000
           BEAV         12113100
1994-02-12 ACME         16245600
           BEAV         14078400
1994-02-13 ACME         17941800
           BEAV         17117100
1994-02-14 ACME         11963700
           BEAV         13848900
1994-02-15 ACME         16709700
           BEAV         16980300
1994-02-16 ACME         16625100
           BEAV         15626400
1994-02-17 ACME         16555200
           BEAV         12876300
1994-02-18 ACME          7427700
           BEAV         15035400

[98 rows x 1 columns]

In [23]:
subset = input_data[0].iloc[:,60:64]
subset.columns = ['dswrf_sfc', 'uswrf_sfc', 'tcolc_eatm', 'spfh_2m']

In [28]:
pd.merge()

dswrf_sfc  uswrf_sfc  tcolc_eatm   spfh_2m
train_dates station                                            
1994-01-01  ACME         340.0       73.0      0.0013  0.003630
            BEAV         320.0       71.0      0.0120  0.002840
1994-01-02  ACME         250.0       49.0      0.0221  0.004520
            BEAV         280.0       58.0      0.0006  0.002140
1994-01-03  ACME         350.0       77.0      0.0025  0.002810
            BEAV         330.0       73.0      0.0014  0.002310
1994-01-04  ACME         350.0       77.0      0.0199  0.002588
            BEAV         330.0       74.0      0.0405  0.002078
1994-01-05  ACME         340.0       70.0      0.1769  0.003700
            BEAV         330.0       73.0      0.0000  0.002940
1994-01-06  ACME         350.0       74.0      0.0002  0.002810
            BEAV         320.0       82.0      0.0187  0.002490
1994-01-07  ACME         360.0       86.0      0.0000  0.001030
            BEAV         340.0       87.0      0.0009  0.002320
1994-01-08  ACME         359.0       79.0      0.0001  0.002040
            BEAV         344.0       90.0      0.0003  0.002190
1994-01-09  ACME         330.0       67.0      0.0147  0.004340
            BEAV         350.0       90.0      0.0010  0.002690
1994-01-10  ACME         260.0       55.0      0.0001  0.004160
            BEAV         290.0       57.0      0.0244  0.003570
1994-01-11  ACME         359.0       77.0      0.0032  0.004510
            BEAV         347.0       76.0      0.0233  0.003940
1994-01-12  ACME         360.0       78.0      0.0226  0.004790
            BEAV         350.0       76.0      0.0015  0.003060
1994-01-13  ACME         370.0       78.0      0.0001  0.003501
            BEAV         350.0       77.0      0.0001  0.002881
1994-01-14  ACME         370.0       79.0      0.0000  0.003340
            BEAV         350.0       77.0      0.0001  0.003330
1994-01-15  ACME         370.0       78.0      0.0026  0.001534
            BEAV         360.0       77.0      0.0026  0.001624
...                        ...        ...         ...       ...
1994-02-15  ACME         507.0      103.0      0.0031  0.003038
            BEAV         503.0      104.0      0.0001  0.002618
1994-02-16  ACME         507.0      103.0      0.0009  0.005650
            BEAV         507.0      105.0      0.0074  0.002750
1994-02-17  ACME         481.0       93.0      0.0408  0.006370
            BEAV         487.0       97.0      0.0921  0.005040
1994-02-18  ACME         287.0       50.0      0.1044  0.008040
            BEAV         474.0       93.0      0.2008  0.004980
1994-02-19  ACME         308.0       56.0      0.0060  0.006380
            BEAV         526.0      109.0      0.0008  0.001750
1994-02-20  ACME         526.0      106.0      0.0063  0.003130
            BEAV         529.0      109.0      0.0014  0.002890
1994-02-21  ACME         235.0       37.0      1.4780  0.005198
            BEAV         296.0       50.0      1.4988  0.004088
1994-02-22  ACME         172.0       33.0      0.0086  0.007010
            BEAV         106.0       40.0      0.5832  0.003270
1994-02-23  ACME         554.0      122.0      0.0000  0.002200
            BEAV         549.0      143.0      0.0002  0.002400
1994-02-24  ACME         548.0      111.0      0.0013  0.004410
            BEAV         547.0      123.0      0.0079  0.002410
1994-02-25  ACME         550.0      110.0      0.0003  0.001220
            BEAV         540.0      122.0      0.0008  0.001080
1994-02-26  ACME         559.0      112.0      0.0009  0.001520
            BEAV         553.0      112.0      0.0000  0.001190
1994-02-27  ACME         546.0      107.0      0.0158  0.003670
            BEAV         552.0      111.0      0.0023  0.003040
1994-02-28  ACME         134.0       20.0      0.2823  0.006910
            BEAV         162.0       27.0      0.1937  0.004210
1994-03-01  ACME         333.0       56.0      0.0025  0.004250
            BEAV         556.0      121.0      0.0017

In [33]:
combined = pd.merge(subset.reset_index(), yvals.reset_index(), left_on=['train_dates', 'station'], right_on=['date', 'location'])
corr_data = combined.drop(['date', 'location'], axis=1).set_index(['train_dates', 'station'])
corr_data = corr_data.astype(float)
corr_data

dswrf_sfc  uswrf_sfc  tcolc_eatm   spfh_2m  total_solar
train_dates station                                                         
1994-01-01  ACME         340.0       73.0      0.0013  0.003630   12384900.0
            BEAV         320.0       71.0      0.0120  0.002840   10116900.0
1994-01-02  ACME         250.0       49.0      0.0221  0.004520   11908500.0
            BEAV         280.0       58.0      0.0006  0.002140    9262800.0
1994-01-03  ACME         350.0       77.0      0.0025  0.002810   12470700.0
            BEAV         330.0       73.0      0.0014  0.002310   10789800.0
1994-01-04  ACME         350.0       77.0      0.0199  0.002588   12725400.0
            BEAV         330.0       74.0      0.0405  0.002078   11816700.0
1994-01-05  ACME         340.0       70.0      0.1769  0.003700   10894800.0
            BEAV         330.0       73.0      0.0000  0.002940    9936300.0
1994-01-06  ACME         350.0       74.0      0.0002  0.002810    6639000.0
            BEAV         320.0       82.0      0.0187  0.002490    2245200.0
1994-01-07  ACME         360.0       86.0      0.0000  0.001030   13244700.0
            BEAV         340.0       87.0      0.0009  0.002320    9877800.0
1994-01-08  ACME         359.0       79.0      0.0001  0.002040   12927900.0
            BEAV         344.0       90.0      0.0003  0.002190   11545200.0
1994-01-09  ACME         330.0       67.0      0.0147  0.004340   12600300.0
            BEAV         350.0       90.0      0.0010  0.002690   11295300.0
1994-01-10  ACME         260.0       55.0      0.0001  0.004160    6406500.0
            BEAV         290.0       57.0      0.0244  0.003570   10785000.0
1994-01-11  ACME         359.0       77.0      0.0032  0.004510   12743400.0
            BEAV         347.0       76.0      0.0233  0.003940   11424900.0
1994-01-12  ACME         360.0       78.0      0.0226  0.004790   10453500.0
            BEAV         350.0       76.0      0.0015  0.003060   11794200.0
1994-01-13  ACME         370.0       78.0      0.0001  0.003501   12985200.0
            BEAV         350.0       77.0      0.0001  0.002881   12149100.0
1994-01-14  ACME         370.0       79.0      0.0000  0.003340   13080000.0
            BEAV         350.0       77.0      0.0001  0.003330   12227700.0
1994-01-15  ACME         370.0       78.0      0.0026  0.001534   11826300.0
            BEAV         360.0       77.0      0.0026  0.001624   10290600.0
...                        ...        ...         ...       ...          ...
1994-02-04  ACME         447.0       92.0      0.0070  0.003332   10506600.0
            BEAV         419.0       86.0      0.1075  0.002372   14513400.0
1994-02-05  ACME         455.0       94.0      0.0007  0.003580   15325500.0
            BEAV         454.0      105.0      0.0002  0.002210   14849400.0
1994-02-06  ACME         467.0       97.0      0.0013  0.003880   15568500.0
            BEAV         463.0       98.0      0.0009  0.002130   14813100.0
1994-02-07  ACME         366.0       66.0      0.1868  0.005590    9486600.0
            BEAV         302.0       51.0      0.1819  0.003400    4878900.0
1994-02-08  ACME         402.0       75.0      0.0478  0.005110    3012000.0
            BEAV         168.0       26.0      0.3945  0.002540    6858900.0
1994-02-09  ACME         372.0       78.0      0.0140  0.000860    7137000.0
            BEAV         478.0      101.0      0.0015  0.000460   15905100.0
1994-02-10  ACME         493.0      103.0      0.0002  0.001595   15829800.0
            BEAV         475.0       99.0      0.0024  0.001485   15714300.0
1994-02-11  ACME         467.0       94.0      0.0000  0.003358   16185000.0
            BEAV         399.0       78.0      0.0028  0.003458   12113100.0
1994-02-12  ACME         494.0      101.0      0.0000  0.002192   16245600.0
            BEAV         496.0      105.0      0.0000  0.001392   14078400.0
1994-02-13  ACME         517.0      108.0      0.0000  0.001844   17941800.0
            BEAV 

dswrf_sfc  uswrf_sfc  tcolc_eatm   spfh_2m  total_solar
train_dates station                                                         
1994-01-01  ACME         340.0       73.0      0.0013  0.003630   12384900.0
            BEAV         320.0       71.0      0.0120  0.002840   10116900.0
1994-01-02  ACME         250.0       49.0      0.0221  0.004520   11908500.0
            BEAV         280.0       58.0      0.0006  0.002140    9262800.0
1994-01-03  ACME         350.0       77.0      0.0025  0.002810   12470700.0
            BEAV         330.0       73.0      0.0014  0.002310   10789800.0
1994-01-04  ACME         350.0       77.0      0.0199  0.002588   12725400.0
            BEAV         330.0       74.0      0.0405  0.002078   11816700.0
1994-01-05  ACME         340.0       70.0      0.1769  0.003700   10894800.0
            BEAV         330.0       73.0      0.0000  0.002940    9936300.0
1994-01-06  ACME         350.0       74.0      0.0002  0.002810    6639000.0
            BEAV         320.0       82.0      0.0187  0.002490    2245200.0
1994-01-07  ACME         360.0       86.0      0.0000  0.001030   13244700.0
            BEAV         340.0       87.0      0.0009  0.002320    9877800.0
1994-01-08  ACME         359.0       79.0      0.0001  0.002040   12927900.0
            BEAV         344.0       90.0      0.0003  0.002190   11545200.0
1994-01-09  ACME         330.0       67.0      0.0147  0.004340   12600300.0
            BEAV         350.0       90.0      0.0010  0.002690   11295300.0
1994-01-10  ACME         260.0       55.0      0.0001  0.004160    6406500.0
            BEAV         290.0       57.0      0.0244  0.003570   10785000.0
1994-01-11  ACME         359.0       77.0      0.0032  0.004510   12743400.0
            BEAV         347.0       76.0      0.0233  0.003940   11424900.0
1994-01-12  ACME         360.0       78.0      0.0226  0.004790   10453500.0
            BEAV         350.0       76.0      0.0015  0.003060   11794200.0
1994-01-13  ACME         370.0       78.0      0.0001  0.003501   12985200.0
            BEAV         350.0       77.0      0.0001  0.002881   12149100.0
1994-01-14  ACME         370.0       79.0      0.0000  0.003340   13080000.0
            BEAV         350.0       77.0      0.0001  0.003330   12227700.0
1994-01-15  ACME         370.0       78.0      0.0026  0.001534   11826300.0
            BEAV         360.0       77.0      0.0026  0.001624   10290600.0
...                        ...        ...         ...       ...          ...
1994-02-04  ACME         447.0       92.0      0.0070  0.003332   10506600.0
            BEAV         419.0       86.0      0.1075  0.002372   14513400.0
1994-02-05  ACME         455.0       94.0      0.0007  0.003580   15325500.0
            BEAV         454.0      105.0      0.0002  0.002210   14849400.0
1994-02-06  ACME         467.0       97.0      0.0013  0.003880   15568500.0
            BEAV         463.0       98.0      0.0009  0.002130   14813100.0
1994-02-07  ACME         366.0       66.0      0.1868  0.005590    9486600.0
            BEAV         302.0       51.0      0.1819  0.003400    4878900.0
1994-02-08  ACME         402.0       75.0      0.0478  0.005110    3012000.0
            BEAV         168.0       26.0      0.3945  0.002540    6858900.0
1994-02-09  ACME         372.0       78.0      0.0140  0.000860    7137000.0
            BEAV         478.0      101.0      0.0015  0.000460   15905100.0
1994-02-10  ACME         493.0      103.0      0.0002  0.001595   15829800.0
            BEAV         475.0       99.0      0.0024  0.001485   15714300.0
1994-02-11  ACME         467.0       94.0      0.0000  0.003358   16185000.0
            BEAV         399.0       78.0      0.0028  0.003458   12113100.0
1994-02-12  ACME         494.0      101.0      0.0000  0.002192   16245600.0
            BEAV         496.0      105.0      0.0000  0.001392   14078400.0
1994-02-13  ACME         517.0      108.0      0.0000  0.001844   17941800.0
            BEAV 